In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from tqdm.notebook import tqdm
import re
import time
import datetime
import requests
from bs4 import BeautifulSoup

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')  # 창 뜨지 않게
chrome_options.add_argument('--no-sandbox')  # gui x
chrome_options.add_argument('--disable-dev-shm-usage')  # /deb/shm 디렉토리를 사용 x 공유메모리 관련
driver = webdriver.Chrome(ChromeDriverManager().install(), options = chrome_options)

In [3]:
crawling_path = '/content/drive/MyDrive/final-project/data/'

In [4]:
def movie_crawling(date, genre, genre_num, page, end_review):
    review_list = []
    for i in tqdm(range(1, page+1)):
        url = f'https://movie.naver.com/movie/sdb/rank/rmovie.naver?sel=pnt&date={date}&tg={genre_num}&page={i}'
        response = requests.get(url)
        time.sleep(1.5)
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        movie_list = []
        re_com = re.compile('\d+')
        for a in soup.find_all('div', class_ = 'tit5'):
            movie_list.append(re_com.findall(a.find('a')['href'])[0])
        for now, movie in enumerate(tqdm(movie_list)):
            url_2 = f'https://movie.naver.com/movie/bi/mi/point.naver?code={movie}#tab'
            driver.get(url_2)
            time.sleep(2)
            try:
                movie_name = driver.find_element(by='xpath', value='//*[@id="content"]/div[1]/div[2]/div[1]/h3/a').text
            except:
                continue
            for num in range(1, end_review):
                review_page = f'https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code={movie}&page={num}'   # 사용자 리뷰들만 있는 웹페이지
                driver.get(review_page)
                time.sleep(1.5)
                try:
                    for i in range(1, 10):
                        score = driver.find_element(by='xpath', value = f'/html/body/div/div/div[5]/ul/li[{i}]/div[1]/em').text
                        review = driver.find_element(by='xpath', value =f'/html/body/div/div/div[5]/ul/li[{i}]/div[2]/p/span').text
                        review_list.append([genre, movie_name, score, review])
                except:
                    break
            # if now!=0 and now%25==0:
            #     print(f'Time:{datetime.datetime.now().strftime("%m-%d %H:%M")}', f'{genre}_{now}_{movie_name} ')
    return review_list

In [5]:
page_dic = {'드라마':(1,23),
            '판타지':(2,3),
            '공포':(3,6),
            '멜로/애정/로맨스':(5,7),
            '모험':(6,3),
            '스릴러':(7,4),
            '느와르':(8,1),
            '다큐멘터리':(10,3),
            '코미디':(11,11),
            '가족':(12,1),
            '미스터리':(13,2),
            '전쟁':(14,1),
            '애니메이션':(15,9),
            '범죄':(16,6),
            '뮤지컬':(17,1),
            'SF':(18,4),
            '액션':(19,16)}

In [6]:
# 한번에 모든 장르 크롤링
# review_max = 100
# date = 20220428
# for genre in tqdm(page_dic):
#     review = movie_crawling(date, genre, page_dic[genre][0], page_dic[genre][1], review_max)
#     review = pd.DataFrame(review, columns = ['genre', 'movie_name', 'score','review'])
#     review.to_csv(crawling_path+ fr'\movie_crawling_{genre}.csv')

In [7]:
# genre 에 원하는 장르 하나씩 크롤링
genre = '드라마'
review_max = 100
date = 20220428
today = datetime.datetime.now().strftime('%y%m%d')
review = movie_crawling(date, genre, page_dic[genre][0], page_dic[genre][1], review_max)
review = pd.DataFrame(review, columns = ['genre', 'movie_name', 'score','review'])
review.to_csv(crawling_path+ fr'\movie_crawling_{genre}_{today}.csv')

  0%|          | 0/23 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

In [ ]:
# genres = ['드라마'] => 재현
# genres = ['멜로/애정/로맨스', '모험', '스릴러', '느와르', '범죄'] => 준호
# genres = ['코미디', '가족', '미스터리', '애니메이션'] => 석창
# genres = ['뮤지컬', '액션', '다큐멘터리', '전쟁'] => 예슬

review_max = 100
date = 20220428
today = datetime.datetime.now().strftime('%y%m%d')
for genre in genres:
    review = movie_crawling(date, genre, page_dic[genre][0], page_dic[genre][1], review_max)
    review = pd.DataFrame(review, columns = ['genre', 'movie_name', 'score','review'])
    review.to_csv(crawling_path+ fr'\movie_crawling_{genre}_{today}.csv')